In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-pdfs:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5365370%2F8920748%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240717%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240717T074218Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daa9444fd660b4808f71b16139d4d7fcbe8efc289d1f9344cf7b0cb07c2ee5cb7849c3a51321d0e709cf8e73caaa449d80f0f637b7d42584d911fee1e899548f8b2a8f311f708c4d3b6d0aac3684cb78f757a076195dab2f12c52cf0cce97b343774561ff906264ae937926ac26d9f0f60034f9ba2f493772c64c9f3cca991231780fdfe2b5a700fb3b36547276617696b19953da7f0e69234e1f25d3337d714885db4fca51d77e41dacfcdf446195436ac8f7ce471e456451dd91542319f22a479e933a658699b8fe68a29b5f615584fc6f0cd3214dc944ce6e1bc687e17d0460b484120a50e43ca7282c51a6a26fb6580cb1b8e0dc7bb902b2ee5a1378aba1d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install chromadb==0.5.3 PyMuPDF==1.24.7 langchain_community==0.2.7 langchain_core==0.2.12 langchain_openai==0.1.14 opencv_python==4.10.0.84 opencv_python_headless==4.7.0.72 pytesseract==0.3.10 python-dotenv==1.0.1 sentence_transformers==3.0.1 tabula_py==2.9.3 tiktoken==0.7.0 tqdm==4.66.4 ragas==0.1.10 faiss-cpu==1.8.0.post1

In [ ]:
pip install --index-url https://download.pytorch.org/whl/nightly/cu118 --pre 'torch==2.4.0.dev20240612+cu118'

In [ ]:
!pip install tiktoken bitsandbytes accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

modelPath="nikravan/glm-4vq"
tokenizer = AutoTokenizer.from_pretrained(modelPath, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    modelPath,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto"
)


In [ ]:
from PIL import Image

query = "Get the text from the given image if it has. If it's a graph then list all the relationships and their meanings in detail. If it's a diagram/table/image, list all the possible knowledge in it with the details."
image = Image.open("/kaggle/input/data-pdfs/image.png").convert('RGB')

def image_to_nl(image):
    inputs = tokenizer.apply_chat_template([{"role": "user", "image": image, "content": query}],
                                           add_generation_prompt=True, tokenize=True, return_tensors="pt",
                                           return_dict=True)

    inputs = inputs.to(device)

    gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        outputs = outputs[:, inputs['input_ids'].shape[1]:]
        return tokenizer.decode(outputs[0])
# image_to_nl(image)

In [ ]:
#@title Dataloader
import fitz
import tabula
import re
import pytesseract
from tqdm import tqdm
import numpy as np
import cv2
import io

def table_to_nlp(table):
    # Convert the table to a list of dictionaries
    table_data = table.to_dict(orient='records')

    # Convert the table data to NLP format
    nlp_data = []
    for row in table_data:
        nlp_row = {}
        for key, value in row.items():
            nlp_row[key] = value
        nlp_data.append(nlp_row)

    return nlp_data

def read_tables(pdf_path, page):
    tables = tabula.read_pdf(pdf_path, pages=page+1, multiple_tables=True)
    table_datas = []
    for table in tables:
        # print(page)
        # print(table.shape)
        if table.shape[0]<2 or table.shape[1]<2:
            # print('skipping')
            continue
        # table = table.dropna(thresh=table.shape[1]-2)
        table_data = table_to_nlp(table)
        table_datas.append({ "table_data":table_data, "columns":list(table.columns)})
    table_text = [ f"This table has {len(table['columns'])} columns which are {table['columns']}.\n Table information in json \n {str(table['table_data'])}" for table in table_datas]
    return table_text


def pdf_to_markdown_ocr(pdf_path):
    document = fitz.open(pdf_path)
    markdown_text = ""
    num_before = 0
    image_docs = []
    table_docs = []

    for page in tqdm(document):
        text = page.get_text("text")

        # Split the text into lines
        lines = text.split('\n')
        skip = 0

        # Text extraction
        for i,line in enumerate(lines):

            if skip > 0:
                skip -= 1
                continue
            # Trim whitespace
            line = line.strip()

            # Check if the line is likely to be a header
            if i<len(lines)-1:
                if is_likely_header_number(line, lines[i+1], num_before):
                    try:
                        num_before = sum([ float(line.strip().split('.')[i])/10**i for i in range(len(line.strip().split()))])
                        # Determine header level based on some heuristics
                        level = determine_header_level(line)
                        markdown_text += f"\n{'#' * level} {line} {lines[i+1]} \n\n"

                        skip = 1
                    except:
                        markdown_text += f"\n# {line} \n\n"
                else:
                    # Regular text, just add it as is
                    markdown_text += line + '  \n'  # Two spaces for line break

        # Table Extraction
        table_text = read_tables(pdf_path, page.number)
        for table in table_text:
#             markdown_text += '\n' + table + '  \n'
            table_docs.append(table)

        # Image Extraction
        image_list = page.get_images(full=True)
        if len(image_list) > 0:
            for image in image_list:
                xref = image[0]
                base_image = document.extract_image(xref)
                if base_image:
                    image_bytes = base_image["image"]
                    np_array = np.frombuffer(image_bytes, dtype=np.uint8)

                    # Decode the image using PIL
                    img = Image.open(io.BytesIO(np_array))
                    text_from_image = image_to_nl(img)
                    # Save the image
                    # image_filename = f"page{page.number + 1}_image{random.randint(1,1000) + 1}.png"
                    # img.save(os.path.join('outmd', image_filename))
                    # print('saving image')

#                     if base_image["colorspace"] == 1:  # Grayscale
#                         np_array = cv2.imdecode(np_array, cv2.IMREAD_GRAYSCALE)
#                     elif base_image["colorspace"] in [3, 4]:  # RGB or CMYK
#                         np_array = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
#                         if base_image["colorspace"] == 4:
#                             np_array = cv2.cvtColor(np_array, cv2.COLOR_CMYK2RGB)

#                     image = np_array
                    # image = self.deskew(np_array)
                    # text_from_image = pytesseract.image_to_string(image)

                    if len(text_from_image.strip())>0:
                        image_docs.append(text_from_image)
                    else:
                        continue
                    markdown_text += '\n' + text_from_image + '  \n'

        markdown_text += '\n'  # Extra newline between pages
    document.close()
    markdown_text = remove_newline(markdown_text)

    return {"text":markdown_text,
            "image":image_docs,
            "table":table_docs}

def remove_newline(text):
    pattern = r'\n([^A-Z])'
    return re.sub(pattern, r'\1', text)

def is_likely_header_number(line, next_line, num_before):
    line = line.strip()
    nums = line.split('.')
    if line == 'Abstract' or line == 'References':
        return True
    try:
        if len(line)<7 and len(nums)<=3 and [nums[i].isdigit() for i in range(len(nums))].count(True) == len(nums) and next_line.strip()[0].isupper():

            if sum([float(nums[i])/10**i for i in range(len(nums))]) > num_before and sum([float(nums[i])/10**i for i in range(len(nums))]) - num_before <= 1:
                num_before = sum([float(nums[i])/10**i for i in range(len(nums))])
                return True
            else:
                False
    except:
        return False

def determine_header_level(line):
    return line.count('.') + 1


In [ ]:
class PDFLoader():
    def __init__(self, filepaths, read_method="hybridMD"):
        self.paths = filepaths
        self.docs = {}
        self.read_method = read_method
        print('Reading all pdfs')
        self.read()

    def get_docs(self):
        return self.docs

    def read(self):
        for path in self.paths:
            print(f'Reading {path}...')
            self.docs[path] = pdf_to_markdown_ocr(path)
        return self.docs

    def save(self):
        for path in self.paths:
            with open(path+'.txt', "w", encoding="utf-8") as f:
                f.write(self.docs[path]['text'])

In [ ]:
filePaths = [
    '/kaggle/input/data-pdfs/attention.pdf',
#     '/kaggle/input/data-pdfs/imagenet.pdf',
#     '/kaggle/input/data-pdfs/rag.pdf'
]

pdf = PDFLoader(filePaths)

In [ ]:
pdf.get_docs()[filePaths[0]]['image']

In [ ]:
import os

os.environ['AZURE_OPENAI_ENDPOINT'] = "https://openai-for-internship-southindia.openai.azure.com/"
os.environ['AZURE_OPENAI_API_KEY'] = "3c749ed024f440269bd40f8472fa8f56"
os.environ['OPENAI_API_VERSION'] = "2023-12-01-preview"
os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'] = "GPT35-turboA"

In [ ]:
from chromadb.utils import embedding_functions
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"))
ef = embedding_functions.OpenAIEmbeddingFunction(
                    # api_key="your-api-key-here",
                    api_base = "https://openai-for-internship-southindia.openai.azure.com/",
                    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
                    api_type="azure",
                    api_version="2023-12-01-preview",
                    model_name="text-embedding3"
        )

In [ ]:
from langchain_core.documents import Document
text_docs = []
image_docs = []
table_docs = []

for path in pdf.paths:
    text = pdf.get_docs()[path]['text']
    images = pdf.get_docs()[path]['image']
    tables = pdf.get_docs()[path]['table']

    text_docs.extend([Document(doc, metadata={'title':doc.split(' ')[0] , "source":path, "type":"text"}) for doc in text.split('\n# ') if doc])

    image_docs.extend([Document(" ".join(doc.split(' ')[:-1]), metadata={'title':doc.split(' ')[0], "source":path, "type":"image"}) for doc in images])

    table_docs.extend([Document(doc, metadata={'title':doc.split(' ')[0], "source":path, "type":"table"}) for doc in tables])


In [ ]:
import matplotlib.pyplot as plt
import tiktoken

def num_tokens(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

docs_texts = [d.page_content.replace('<|endoftext|>',' ') for d in text_docs]
# docs_texts.extend([d.page_content.replace('<|endoftext|>',' ') for d in image_docs])

In [ ]:
counts = [num_tokens(d, "cl100k_base") for d in docs_texts]

# Plot histogram of token counts
plt.figure(figsize=(10, 6))
plt.hist(counts, bins=30, color="blue", edgecolor="black", alpha=0.7)
plt.title("Histogram of Token Counts")
plt.xlabel("Token Count")
plt.ylabel("Frequency")
plt.grid(axis="y", alpha=0.75)

# Display the histogram
plt.show()

In [ ]:
# Doc texts split
from langchain_text_splitters import RecursiveCharacterTextSplitter

chunk_size_tok = 400
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size_tok, chunk_overlap=30
)

texts_split = []
for text in docs_texts:
    texts_split.extend(text_splitter.split_text(text))


#
print(f"Number of text splits generated: {len(texts_split)}")


In [ ]:
texts_split.extend([d.page_content for d in table_docs])
docs_texts.extend([d.page_content for d in table_docs])

In [ ]:
global_embeddings = ef(texts_split)

In [ ]:
import matplotlib.pyplot as plt
from typing import Optional
import numpy as np
from umap import UMAP

def reduce_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


dim = 2
global_embeddings_reduced = reduce_cluster_embeddings(global_embeddings, dim)
print(global_embeddings_reduced[0])
#
plt.figure(figsize=(10, 8))
plt.scatter(global_embeddings_reduced[:, 0], global_embeddings_reduced[:, 1], alpha=0.5)
plt.title("Global Embeddings")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture

def get_optimal_clusters(embeddings: np.ndarray, max_clusters: int = 50, random_state: int = 1234):
    max_clusters = min(max_clusters, len(embeddings))
    bics = [GaussianMixture(n_components=n, random_state=random_state).fit(embeddings).bic(embeddings)
            for n in range(1, max_clusters)]
    return np.argmin(bics) + 1

def gmm_clustering(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state).fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters
#
labels, _ = gmm_clustering(global_embeddings_reduced, threshold=0.5)

plot_labels = np.array([label[0] if len(label) > 0 else -1 for label in labels])
plt.figure(figsize=(10, 8))

unique_labels = np.unique(plot_labels)
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))

for label, color in zip(unique_labels, colors):
    mask = plot_labels == label
    plt.scatter(global_embeddings_reduced[mask, 0], global_embeddings_reduced[mask, 1], color=color, label=f'Cluster {label}', alpha=0.5)

plt.title("Cluster Visualization of Embeddings")
plt.xlabel("Dim1")
plt.ylabel("Dim2")
plt.legend()
plt.show()


In [ ]:
import pandas as pd

simple_labels = [label[0] if len(label) > 0 else -1 for label in labels]

df = pd.DataFrame({
    'Text': texts_split,
    'Embedding': list(global_embeddings_reduced),
    'Cluster': simple_labels
})

def format_cluster_texts(df):
    clustered_texts = {}
    for cluster in df['Cluster'].unique():
        cluster_texts = df[df['Cluster'] == cluster]['Text'].tolist()
        clustered_texts[cluster] = " --- ".join(cluster_texts)
    return clustered_texts
clustered_texts = format_cluster_texts(df)


In [ ]:
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import umap
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.mixture import GaussianMixture

RANDOM_SEED = 2

def global_cluster_embeddings(
    embeddings: np.ndarray,
    dim: int,
    n_neighbors: Optional[int] = None,
    metric: str = "cosine",
) -> np.ndarray:
    """
    Perform global dimensionality reduction on the embeddings using UMAP.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - n_neighbors: Optional; the number of neighbors to consider for each point.
                   If not provided, it defaults to the square root of the number of embeddings.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    if n_neighbors is None:
        n_neighbors = int((len(embeddings) - 1) ** 0.5)
    return umap.UMAP(
        n_neighbors=n_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def local_cluster_embeddings(
    embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
) -> np.ndarray:
    """
    Perform local dimensionality reduction on the embeddings using UMAP, typically after global clustering.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for the reduced space.
    - num_neighbors: The number of neighbors to consider for each point.
    - metric: The distance metric to use for UMAP.

    Returns:
    - A numpy array of the embeddings reduced to the specified dimensionality.
    """
    return umap.UMAP(
        n_neighbors=num_neighbors, n_components=dim, metric=metric
    ).fit_transform(embeddings)


def get_optimal_clusters(
    embeddings: np.ndarray, max_clusters: int = 50, random_state: int = RANDOM_SEED
) -> int:
    """
    Determine the optimal number of clusters using the Bayesian Information Criterion (BIC) with a Gaussian Mixture Model.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - max_clusters: The maximum number of clusters to consider.
    - random_state: Seed for reproducibility.

    Returns:
    - An integer representing the optimal number of clusters found.
    """
    max_clusters = min(max_clusters, len(embeddings))
    n_clusters = np.arange(1, max_clusters)
    bics = []
    for n in n_clusters:
        gm = GaussianMixture(n_components=n, random_state=random_state)
        gm.fit(embeddings)
        bics.append(gm.bic(embeddings))
    return n_clusters[np.argmin(bics)]


def GMM_cluster(embeddings: np.ndarray, threshold: float, random_state: int = 0):
    """
    Cluster embeddings using a Gaussian Mixture Model (GMM) based on a probability threshold.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - threshold: The probability threshold for assigning an embedding to a cluster.
    - random_state: Seed for reproducibility.

    Returns:
    - A tuple containing the cluster labels and the number of clusters determined.
    """
    n_clusters = get_optimal_clusters(embeddings)
    gm = GaussianMixture(n_components=n_clusters, random_state=random_state)
    gm.fit(embeddings)
    probs = gm.predict_proba(embeddings)
    labels = [np.where(prob > threshold)[0] for prob in probs]
    return labels, n_clusters


def perform_clustering(
    embeddings: np.ndarray,
    dim: int,
    threshold: float,
) -> List[np.ndarray]:
    """
    Perform clustering on the embeddings by first reducing their dimensionality globally, then clustering
    using a Gaussian Mixture Model, and finally performing local clustering within each global cluster.

    Parameters:
    - embeddings: The input embeddings as a numpy array.
    - dim: The target dimensionality for UMAP reduction.
    - threshold: The probability threshold for assigning an embedding to a cluster in GMM.

    Returns:
    - A list of numpy arrays, where each array contains the cluster IDs for each embedding.
    """
    if len(embeddings) <= dim + 1:
        # Avoid clustering when there's insufficient data
        return [np.array([0]) for _ in range(len(embeddings))]

    # Global dimensionality reduction
    reduced_embeddings_global = global_cluster_embeddings(embeddings, dim)
    # Global clustering
    global_clusters, n_global_clusters = GMM_cluster(
        reduced_embeddings_global, threshold
    )

    all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
    total_clusters = 0

    # Iterate through each global cluster to perform local clustering
    for i in range(n_global_clusters):
        # Extract embeddings belonging to the current global cluster
#         global_cluster_embeddings_ = embeddings[
#             np.array([i in gc for gc in global_clusters])
#         ]

        global_cluster_mask = np.array([i in gc for gc in global_clusters])
        global_cluster_embeddings_ = embeddings[global_cluster_mask]

        if len(global_cluster_embeddings_) == 0:
            continue
        if len(global_cluster_embeddings_) <= dim + 1:
            # Handle small clusters with direct assignment
            local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
            n_local_clusters = 1
        else:
            # Local dimensionality reduction and clustering
            reduced_embeddings_local = local_cluster_embeddings(
                global_cluster_embeddings_, dim
            )
            local_clusters, n_local_clusters = GMM_cluster(
                reduced_embeddings_local, threshold
            )

        # Assign local cluster IDs, adjusting for total clusters already processed
        for j in range(n_local_clusters):
            local_cluster_embeddings_ = global_cluster_embeddings_[
                np.array([j in lc for lc in local_clusters])
            ]
            indices = np.where(
                (embeddings == local_cluster_embeddings_[:, None]).all(-1)
            )[1]
            for idx in indices:
                all_local_clusters[idx] = np.append(
                    all_local_clusters[idx], j + total_clusters
                )

        total_clusters += n_local_clusters

    return all_local_clusters


def embed(texts):
    """
    Generate embeddings for a list of text documents.

    This function assumes the existence of an `embd` object with a method `embed_documents`
    that takes a list of texts and returns their embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be embedded.

    Returns:
    - numpy.ndarray: An array of embeddings for the given text documents.
    """
    text_embeddings = embd.embed_documents(texts)
    text_embeddings_np = np.array(text_embeddings)
    return text_embeddings_np


def embed_cluster_texts(texts):
    """
    Embeds a list of texts and clusters them, returning a DataFrame with texts, their embeddings, and cluster labels.

    This function combines embedding generation and clustering into a single step. It assumes the existence
    of a previously defined `perform_clustering` function that performs clustering on the embeddings.

    Parameters:
    - texts: List[str], a list of text documents to be processed.

    Returns:
    - pandas.DataFrame: A DataFrame containing the original texts, their embeddings, and the assigned cluster labels.
    """
    text_embeddings_np = ef(texts)  # Generate embeddings
    cluster_labels = perform_clustering(
        text_embeddings_np, 16, 0.1
    )  # Perform clustering on the embeddings
    df = pd.DataFrame()  # Initialize a DataFrame to store the results
    df["text"] = texts  # Store original texts
    df["embd"] = list(text_embeddings_np)  # Store embeddings as a list in the DataFrame
    df["cluster"] = cluster_labels  # Store cluster labels
    return df


def fmt_txt(df: pd.DataFrame) -> str:
    """
    Formats the text documents in a DataFrame into a single string.

    Parameters:
    - df: DataFrame containing the 'text' column with text documents to format.

    Returns:
    - A single string where all text documents are joined by a specific delimiter.
    """
    unique_txt = df["text"].tolist()
    return "--- --- \n --- --- ".join(unique_txt)


def embed_cluster_summarize_texts(
    texts: List[str], level: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Embeds, clusters, and summarizes a list of texts. This function first generates embeddings for the texts,
    clusters them based on similarity, expands the cluster assignments for easier processing, and then summarizes
    the content within each cluster.

    Parameters:
    - texts: A list of text documents to be processed.
    - level: An integer parameter that could define the depth or detail of processing.

    Returns:
    - Tuple containing two DataFrames:
      1. The first DataFrame (`df_clusters`) includes the original texts, their embeddings, and cluster assignments.
      2. The second DataFrame (`df_summary`) contains summaries for each cluster, the specified level of detail,
         and the cluster identifiers.
    """

    # Embed and cluster the texts, resulting in a DataFrame with 'text', 'embd', and 'cluster' columns
    df_clusters = embed_cluster_texts(texts)

    # Prepare to expand the DataFrame for easier manipulation of clusters
    expanded_list = []

    # Expand DataFrame entries to document-cluster pairings for straightforward processing
    for index, row in df_clusters.iterrows():
        for cluster in row["cluster"]:
            expanded_list.append(
                {"text": row["text"], "embd": row["embd"], "cluster": cluster}
            )

    # Create a new DataFrame from the expanded list
    expanded_df = pd.DataFrame(expanded_list)

    # Retrieve unique cluster identifiers for processing
    all_clusters = expanded_df["cluster"].unique()

    print(f"--Generated {len(all_clusters)} clusters--")

    # Summarization
    template = """Here is a sub-set of LangChain Expression Langauge doc.

    LangChain Expression Langauge provides a way to compose chain in LangChain.

    Give a detailed summary of the documentation provided.

    Documentation:
    {context}
    """
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm | StrOutputParser()

    # Format text within each cluster for summarization
    summaries = []
    for i in all_clusters:
        df_cluster = expanded_df[expanded_df["cluster"] == i]
        formatted_txt = fmt_txt(df_cluster)
        summaries.append(chain.invoke({"context": formatted_txt}))

    # Create a DataFrame to store summaries with their corresponding cluster and level
    df_summary = pd.DataFrame(
        {
            "summaries": summaries,
            "level": [level] * len(summaries),
            "cluster": list(all_clusters),
        }
    )

    return df_clusters, df_summary


def recursive_embed_cluster_summarize(
    texts: List[str], level: int = 1, n_levels: int = 3
) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Recursively embeds, clusters, and summarizes texts up to a specified level or until
    the number of unique clusters becomes 1, storing the results at each level.

    Parameters:
    - texts: List[str], texts to be processed.
    - level: int, current recursion level (starts at 1).
    - n_levels: int, maximum depth of recursion.

    Returns:
    - Dict[int, Tuple[pd.DataFrame, pd.DataFrame]], a dictionary where keys are the recursion
      levels and values are tuples containing the clusters DataFrame and summaries DataFrame at that level.
    """
    results = {}  # Dictionary to store results at each level

    # Perform embedding, clustering, and summarization for the current level
    df_clusters, df_summary = embed_cluster_summarize_texts(texts, level)

    # Store the results of the current level
    results[level] = (df_clusters, df_summary)

    # Determine if further recursion is possible and meaningful
    unique_clusters = df_summary["cluster"].nunique()
    if level < n_levels and unique_clusters > 1:
        # Use summaries as the input texts for the next level of recursion
        new_texts = df_summary["summaries"].tolist()
        next_level_results = recursive_embed_cluster_summarize(
            new_texts, level + 1, n_levels
        )

        # Merge the results from the next level into the current results dictionary
        results.update(next_level_results)

    return results

In [ ]:
len(docs_texts)

In [ ]:
leaf_texts = docs_texts
results = recursive_embed_cluster_summarize(leaf_texts, level=1, n_levels=3)

In [ ]:
# Initialize all_texts with leaf_texts
all_texts = leaf_texts.copy()

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)
#Final Summaries extracted
# print(all_texts)

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings

embeddings_azure= AzureOpenAIEmbeddings(model='text-embedding3')

# Now, use all_texts to build the vectorstore with Chroma
vectorstore = Chroma.from_texts(texts=all_texts, embedding=embeddings_azure, )
retriever = vectorstore.as_retriever()

In [ ]:
from time import sleep
def variant_rag_answer(query, vectorstore = vectorstore):
    global context
    answer_chain = PromptTemplate.from_template("Question:{query}\nGiven the context:{context}.Answer the query. DON'T ANSWER ANYTHING OUT OF CONTEXT. IF YOU DON'T KNOW THE ANSWER SAY I DON'T KNOW.") | llm | StrOutputParser()

    variant_chain = PromptTemplate.from_template("Given the query:{query}.Write 3 variants of the query separated by newline.") | llm | StrOutputParser()

    compressor_chain = PromptTemplate.from_template("Given the query:{query} and the context:{context}\nFind and remove the irrelevant sentences from the context. DON'T CHANGE THE RELEVANT SENTENCES.") | llm | StrOutputParser()

    query_variants = variant_chain.invoke({"query":query}).split('\n')

    chunks = []
    for q in query_variants:
        chunks = vectorstore._similarity_search_with_relevance_scores(q, k=5)

    doc_scores = {}
    for q in query_variants:
        chunks = vectorstore._similarity_search_with_relevance_scores(q, k=10)
        for chunk in chunks:
            if chunk[0].page_content in doc_scores:
                doc_scores[chunk[0].page_content] += float(chunk[1])
            else:
                doc_scores[chunk[0].page_content] = float(chunk[1])
    # sort dict with values get the top scorers
    sorted_scores = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)
    sorted_passages = [s[0] for s in sorted_scores[:3]]

    con = "\n".join(sorted_passages)
    con = compressor_chain.invoke({"query":query, "context":con})
    context = con
    ans = ''
    while not ans:
        try:
            ans = answer_chain.invoke({"query":query, "context":con})
        except:
            sleep(2)
            continue
    return ans


In [ ]:
question = "Who are the authors of attention paper?"

In [ ]:
if query_to_be_splitted(question):
        print("Splitting...")
        split_queries = split_query(question)
        split_queries.append(question)
        print("Split queries:", len(split_queries))
        split_answers = []
        for q in split_queries:
            split_answer = variant_rag_answer(q)
            split_answers.append(split_answer)
        rag_contexts.append("\n".join(split_answers))
        answer = answer_chain.invoke({"query":question, "context":"\n".join(split_answers)})
        answers.append(answer)
else:
    print("Not splitting...")
    answer = variant_rag_answer(question)
    rag_contexts.append(context)
    answers.append(answer)

